# Setup

In [4]:
import pandas as pd

In [16]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

# Vector search

In [32]:
books = pd.read_csv("data/books_cleaned.csv")

In [34]:
books['tagged_description'].to_csv('data/tagged_description.txt', index=False, header=False)

In [13]:
raw_docs = TextLoader("data/tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
docs = text_splitter.split_documents(raw_docs)

Created a chunk of size 1170, which is longer than the specified 1
Created a chunk of size 1216, which is longer than the specified 1
Created a chunk of size 375, which is longer than the specified 1
Created a chunk of size 311, which is longer than the specified 1
Created a chunk of size 483, which is longer than the specified 1
Created a chunk of size 484, which is longer than the specified 1
Created a chunk of size 962, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 845, which is longer than the specified 1
Created a chunk of size 296, which is longer than the specified 1
Created a chunk of size 197, which is longer than the specified 1
Created a chunk of size 881, which is longer than the specified 1
Created a chunk of size 1090, which is longer than the specified 1
Created a chunk of size 1191, which is longer than the specified 1
Created a chunk of size 306, which is longer than the specified 1
Create

In [18]:
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
db_books = Chroma.from_documents(docs, embedding=hf_embeddings, persist_directory="./chroma_db")

# Querying the vector database

In [48]:
q = "A book to teach children about nature"

In [87]:
def get_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=top_k)
    books_list = []
    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)][['title', 'authors', 'categories', 'description', 'average_rating', 'num_pages', 'published_year']]

In [88]:
r = get_recommendations(q)

In [89]:
r

,title,authors,categories,description,average_rating,num_pages,published_year
324,Prodigal Summer,Barbara Kingsolver,Fiction,Barbara Kingsolver's fifth novel is a hymn to ...,4.00,444.0,2001.0
404,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,"‘Racso, a brash and boastful little rodent, is...",3.76,288.0,1988.0
406,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,"When Margaret and her younger brother, Artie, ...",3.52,272.0,1991.0
1642,The Control of Nature,John McPhee,Nature,The Control of Nature is John McPhee's bestsel...,4.24,288.0,1990.0
3747,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,Children will discover the exciting world of t...,3.89,16.0,2001.0
3748,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,"Introducing your baby to birds, cats, dogs, an...",3.78,20.0,2002.0
3749,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,"Introduce your babies to birds, cats, dogs, an...",4.03,20.0,2002.0
3750,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,"Introduce your baby to birds, cats, dogs, and ...",3.81,20.0,2002.0
3765,"Baby Einstein: Water, Water Everywhere","Disney Book Group,",Juvenile Fiction,Charming illustrations and playful rhythmic ve...,3.70,10.0,2003.0
3834,The 10 Commandments of Parenting,H. Edwin Young,Religion,Drawn from years of counseling and the author'...,4.00,224.0,2005.0
